In [1]:
import sys

sys.path.append("./utils/")

from config import MINIMUM_GAMEPLAY_COLUMNS
from config import KEEP_COLS_PT
from datetime import timedelta
from get_delta_dna_data import execute_query

In [2]:
file_path = 'sql/events_live_3.6+.sql'
with open(file_path, "r") as f:
    query = f.read()

columns = MINIMUM_GAMEPLAY_COLUMNS

In [3]:
df = execute_query(query, columns)

In [4]:
df.shape

(2150, 12)

In [5]:
df.head()

,userid,childid,eventname,eventtimestamp,mssincelastevent,timezoneoffset,gausercountry,clientversion,pnsmode,pnsword,pnsduration,pnsmistakes
0,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,areaSummary,2023-02-17 19:04:35.709,129385.0,+1300,NZ,3.6.0,None,None,NaN,NaN
1,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,onboarding,2023-02-16 06:22:37.563,8094.0,+1300,NZ,3.6.0,None,None,NaN,NaN
2,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,pnsCompleted,2023-02-16 06:29:48.964,83767.0,+1300,NZ,3.6.0,MatchCollocations,3e493be8-9625-4bfa-af80-4b90632464b7,12.159393,0.0
3,00355778-afa1-4d23-bf22-5968bdd46509,None,onboarding,2023-02-16 06:30:26.866,2192.0,+1300,NZ,3.6.0,None,None,NaN,NaN
4,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,pnsCompleted,2023-02-16 06:24:01.213,20068.0,+1300,NZ,3.6.0,AssembleSyllables,3e493be8-9625-4bfa-af80-4b90632464b7,20.082725,0.0


In [6]:
events_live_data = df.copy()

In [7]:
# events_live_data = events_live_data[~events_live_data['timezoneoffset'].isnull()]

In [8]:
# Adjusted Date Times CTE
events_live_data = events_live_data[~events_live_data['timezoneoffset'].isnull()]
adjusted_date_times = events_live_data.copy()
adjusted_date_times["local_timestamp"] = adjusted_date_times.apply(
    lambda row: row["eventtimestamp"] + timedelta(
        minutes=int(row["timezoneoffset"][:3]) * 60 + int(row["timezoneoffset"][3:5])
    ) if len(row["timezoneoffset"]) > 0 else row["eventtimestamp"],
    axis=1
)
adjusted_date_times["local_eventdate"] = adjusted_date_times["local_timestamp"].dt.floor("D")
adjusted_date_times["local_startdate"] = adjusted_date_times.groupby("userid")["local_eventdate"].transform("min")

# Play Time CTE
play_time = adjusted_date_times.sort_values(['childid', 'local_timestamp']) .copy()

play_time["minutes_played"] = (
    play_time.groupby("childid")["mssincelastevent"]
    .cumsum()
    .div(60000)
)

# Calculate the "exposures" column
completed = adjusted_date_times.copy()
completed = completed[completed["eventname"] == "pnsCompleted"]
completed = completed[~completed["pnsmode"].str.startswith("Pop")]

# Sort the DataFrame by childid, local_timestamp, and pnsword
completed = completed.sort_values(["childid", "local_timestamp", "pnsword"])

# Calculate the "exposures" column based on word changes
completed["exposures"] = (
    completed.groupby(["childid", "pnsword"])
    .cumcount()
    .add(1)
)


# Calculate day_exp and day_since_start
completed["day_exp"] = completed.groupby(["childid", "pnsword"])["local_timestamp"].transform(
    lambda x: (x - x.iloc[0]).dt.days
)
completed["day_since_start"] = completed["local_eventdate"] - completed["local_startdate"]


# Merge with play_time
result = completed.merge(
    play_time[["childid", "local_timestamp", "minutes_played"]],
    on=["childid", "local_timestamp"]
)

# Keep only the columns we need
result = result[KEEP_COLS_PT]

result[:25]

,userid,childid,local_eventdate,local_timestamp,local_startdate,gausercountry,clientversion,pnsmode,pnsword,pnsduration,pnsmistakes,exposures,day_since_start,minutes_played
0,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-04-29,2023-04-29 11:46:13.048,2023-04-04,US,3.7.1,AssembleSyllables,14d3bc08-9522-4db3-8b43-f5d07af8c0be,32.631165,0.0,1,25 days,5.783867
1,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-04-29,2023-04-29 11:47:02.291,2023-04-04,US,3.7.1,AssembleSyllables,b7e25d09-fa97-4ec0-b78f-74970235d2d8,16.949310,1.0,1,25 days,6.604583
2,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-04-29,2023-04-29 11:47:36.892,2023-04-04,US,3.7.1,AssembleSyllables,68734481-9483-4301-bab8-1fba42f84b11,5.428284,1.0,1,25 days,7.181267
3,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-04-29,2023-04-29 11:50:30.621,2023-04-04,US,3.7.1,MatchSynonyms,14d3bc08-9522-4db3-8b43-f5d07af8c0be,4.562683,1.0,2,25 days,10.044583
4,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-04-29,2023-04-29 11:52:03.504,2023-04-04,US,3.7.1,MatchCollocations,b7e25d09-fa97-4ec0-b78f-74970235d2d8,9.725037,1.0,2,25 days,11.592633
5,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-05-04,2023-05-04 17:14:22.349,2023-04-04,US,3.8.0,AssembleSyllables,14d3bc08-9522-4db3-8b43-f5d07af8c0be,5.096115,0.0,3,30 days,16.367333
6,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-05-04,2023-05-04 17:15:20.881,2023-04-04,US,3.8.0,AssembleSyllables,b7e25d09-fa97-4ec0-b78f-74970235d2d8,4.660965,0.0,3,30 days,17.342867
7,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-05-04,2023-05-04 17:15:41.060,2023-04-04,US,3.8.0,MatchSynonyms,68734481-9483-4301-bab8-1fba42f84b11,5.094254,2.0,2,30 days,17.679183
8,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-05-04,2023-05-04 17:16:58.640,2023-04-04,US,3.8.0,AssembleSyllables,a632a290-7172-47a1-b04a-5f9caf7a6676,6.126923,1.0,1,30 days,18.972183
9,000ad428-f561-4586-b6bb-6587e6e39832,056a6862697766655a23e8a6ece42ac27d6747fb,2023-05-04,2023-05-04 17:17:56.546,2023-04-04,US,3.8.0,AssembleSyllables,dfaf8c45-20e7-45b5-9961-7701c08fab6a,16.115173,7.0,1,30 days,19.937283


In [13]:
result[result['gausercountry']=='NZ']

,userid,childid,local_eventdate,local_timestamp,local_startdate,gausercountry,clientversion,pnsmode,pnsword,pnsduration,pnsmistakes,exposures,day_since_start,minutes_played
62,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-16,2023-02-16 19:24:01.213,2023-02-16,NZ,3.6.0,AssembleSyllables,3e493be8-9625-4bfa-af80-4b90632464b7,20.082725,0.0,1,0 days,2.013667
63,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-16,2023-02-16 19:24:45.942,2023-02-16,NZ,3.6.0,AssembleSyllables,39a8e515-c95a-4204-bfd9-ba666c78b8b5,7.106354,0.0,1,0 days,2.759150
64,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-16,2023-02-16 19:25:31.115,2023-02-16,NZ,3.6.0,AssembleSyllables,1f8141da-02a3-416a-b5bc-31561c5061d4,4.753983,0.0,1,0 days,3.512033
65,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-16,2023-02-16 19:29:48.964,2023-02-16,NZ,3.6.0,MatchCollocations,3e493be8-9625-4bfa-af80-4b90632464b7,12.159393,0.0,2,0 days,7.749400
66,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-17,2023-02-17 16:26:56.485,2023-02-16,NZ,3.6.0,AssembleSyllables,3e493be8-9625-4bfa-af80-4b90632464b7,4.437302,0.0,3,1 days,10.735900
67,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-17,2023-02-17 16:28:29.279,2023-02-16,NZ,3.6.0,MatchSynonyms,1f8141da-02a3-416a-b5bc-31561c5061d4,12.329132,0.0,2,1 days,12.282467
68,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-17,2023-02-17 16:29:55.285,2023-02-16,NZ,3.6.0,MatchSynonyms,39a8e515-c95a-4204-bfd9-ba666c78b8b5,7.709412,0.0,2,1 days,13.715900
69,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-17,2023-02-17 16:34:30.422,2023-02-16,NZ,3.6.0,AssembleSyllables,179731c3-1977-454f-a97f-9c8f2a6087bd,5.738892,0.0,1,1 days,18.301517
70,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-18,2023-02-18 08:05:51.283,2023-02-16,NZ,3.6.0,AssembleSyllables,1f8141da-02a3-416a-b5bc-31561c5061d4,4.245758,0.0,3,2 days,31.080300
71,00355778-afa1-4d23-bf22-5968bdd46509,21331651e10312f1dd8741858d339d0b7401a983,2023-02-18,2023-02-18 08:09:44.629,2023-02-16,NZ,3.6.0,AssembleSyllables,39a8e515-c95a-4204-bfd9-ba666c78b8b5,6.639069,0.0,3,2 days,34.969400
